In [2]:
import os
import pickle

import numpy as np
import pandas as pd
import arviz as az
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import pystan
%load_ext jupyterstan

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
STAN_MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "stan_models")
STAN_DATA_PATH = os.path.join(PROJECT_ROOT_DIR,'data')

from stan_utils import *

In [ ]:
%%stan matern_cov_error
functions {
  matrix cov_matern32(vector[] x, real alpha, real phi) {
    int N=size(x);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in i:N){
          cov[i,j] = square(alpha) * (1 + distance(x[i],x[j]) / phi) * exp(-1*distance(x[i], x[j]) / phi);
          cov[j,i] = cov[i, j];
      }
    }    
    return cov;
  }
}

data {
  int<lower=1> D; //数据组数
  int<lower=1> N; //感知节点个数
  vector[N] recv_x;// 每组数据有N个感知节点
  vector[N] recv_y;// 
  real tran_power[D]; //信号源发送功率
  real tran_x[D]; //信号源坐标
  real tran_y[D];
  vector[N] tran_recv_distance[D];
  vector[N] Y[D];
 
}

transformed data {
  matrix[N, N] distance_between_recv;
  vector[2] points[N];
  for (i in 1:N){
    for (j in 1:N){
      distance_between_recv[i,j] = sqrt((recv_x[i]-recv_x[j])^2 + (recv_y[i]-recv_y[j])^2);
    }
  }
  for (i in 1:N) {
    points[i][1] = recv_x[i];
    points[i][2] = recv_y[i];
  }
}

parameters {
  real<lower=0> gama; // 自由空间传播损耗因子
  real<lower=0> fai;  // matern 协方差函数的距离 参数
  real<lower=0> sigma; // 空间方差  
  real<lower=0> sigma_eta; // 测量误差
}
transformed parameters {


}

model {
  vector[N] mu[D];
  matrix[N,N] bigSigma;
  matrix[N,N] L_K;
  real sigma_eta2;
   
  /********************/
  /*    指定先验分布    */
  /********************/
  gama ~ normal(4, 4);
  fai ~ gamma(1, 0.01);
  sigma ~ inv_gamma(2, 10);
  sigma_eta ~ inv_gamma(2, 10);
  
 
  bigSigma = cov_matern32(points, sigma, fai);
  
  sigma_eta2 = square(sigma_eta);
  for (n in 1:N) {
    bigSigma[n, n] = bigSigma[n, n] + sigma_eta2;
  }
  
  L_K = cholesky_decompose(bigSigma);
   
  for (i in 1:D){
    mu[i] = tran_power[i] - 10 * gama * log10(tran_recv_distance[i]);
    Y[i] ~ multi_normal_cholesky(mu[i], L_K); 
  }
   
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL matern_cov_error_482edcc8f642041b3f7dadd82a3038d2 NOW.


Creating pystan model & assigning it to variable name "matern_cov_error".
Stan options:
 {'model_name': 'matern_cov_error', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}


In [15]:
StanModel_cache(matern_cov_error, 'matern_cov_error')

Model cached as:matern_cov_error.pkl


In [5]:
%%stan matern_cov_error_reparameter
functions {
  matrix cov_matern32(vector[] x, real alpha, real phi) {
    int N=size(x);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in i:N){
          cov[i,j] = square(alpha) * (1 + distance(x[i],x[j]) / phi) * exp(-1*distance(x[i], x[j]) / phi);
          cov[j,i] = cov[i, j];
      }
    }    
    return cov;
  }
}

data {
  int<lower=1> D; //数据组数
  int<lower=1> N; //感知节点个数
  vector[N] recv_x;// 每组数据有N个感知节点
  vector[N] recv_y;// 
  real tran_power[D]; //信号源发送功率
  real tran_x[D]; //信号源坐标
  real tran_y[D];
  vector[N] tran_recv_distance[D];
  vector[N] Y[D];
 
}

transformed data {
  matrix[N, N] distance_between_recv;
  vector[2] points[N];
  for (i in 1:N){
    for (j in 1:N){
      distance_between_recv[i,j] = sqrt((recv_x[i]-recv_x[j])^2 + (recv_y[i]-recv_y[j])^2);
    }
  }
  for (i in 1:N) {
    points[i][1] = recv_x[i];
    points[i][2] = recv_y[i];
  }
}

parameters {
  real<lower=0> gama; // 自由空间传播损耗因子
  real<lower=0> alpha;  // 
  real<lower=0> eta; // 空间方差  
  real<lower=0> sigma_eta; // 测量误差
}
transformed parameters {
  real<lower=0> fai;
  real<lower=0> sigma;
  fai = 2 * sqrt(3/2) / alpha;
  sigma = sqrt(fai^3 * eta);
}

model {
  vector[N] mu[D];
  matrix[N,N] bigSigma;
  matrix[N,N] L_K;
  real sigma_eta2;
   
  /********************/
  /*    指定先验分布    */
  /********************/
  gama ~ normal(4, 4);
  alpha ~ gamma(1, 0.01);
  eta ~ inv_gamma(2, 10);
  sigma_eta ~ inv_gamma(2, 10);
  
 
  bigSigma = cov_matern32(points, sigma, fai);
  
  sigma_eta2 = square(sigma_eta);
  for (n in 1:N) {
    bigSigma[n, n] = bigSigma[n, n] + sigma_eta2;
  }
  
  L_K = cholesky_decompose(bigSigma);
   
  for (i in 1:D){
    mu[i] = tran_power[i] - 10 * gama * log10(tran_recv_distance[i]);
    Y[i] ~ multi_normal_cholesky(mu[i], L_K); 
  }
   
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL matern_cov_error_reparameter_22fa18ebd80a802ffcc7b6afba46cae2 NOW.


Creating pystan model & assigning it to variable name "matern_cov_error_reparameter".
Stan options:
 {'model_name': 'matern_cov_error_reparameter', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}
StanModel now available as variable "matern_cov_error_reparameter"!
Compilation took a minute.


In [6]:
StanModel_cache(matern_cov_error_reparameter, 'matern_cov_error_reparameter')

Model cached as:matern_cov_error_reparameter.pkl
